# Chapter 10: Introduction to Artificial Neural Networks with Keras

In [ ]:
import keras_tuner as kt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

plt.style.use("dark_background")

1. [TensorFlow playground](https://playground.tensorflow.org/)

2. Drawing of original ANN (as in figure 10-3) that computes, XOR: $A\oplus B$. Assuming that a neuron is activated when $\geq 2$ of its input connections are active.

<img width="20%" src="datasets/IMG_B049F35CA3D0-1.jpeg"/><img width="20%" src="datasets/IMG_BBB9A8B61AFF-1.jpeg"/>

3. Logistic regression & a classic perceptron, made up of a single layer of threshold logic units (LTUs), function very similarly as classifiers. They differ in two regards:
    * Logistic regression uses the sigmoid activation function (or softmax), whereas the perceptron uses a step function. The sigmoid has the benefit of outputting a probability rather than a 1 or 0. It also has a non-zero gradient, which means it can be trained via gradient descent.
    * They also differ by their training algorithms. The perceptron training algorithm (a.k.a. Hebian learning) will only converge if the training instances are linearly separable, whereas gradient descent (or SGD) will.

4. The sigmoid (or any non-linear) activation function is a key ingredient in training MLPs because it has a non-zero gradient. Meaning MLPs can be trained via gradient descent.

5. Sigmoid, Hyperbolic tan, & Rectified Linear Unit are all popular activation functions.

In [ ]:
activations = [
    tf.keras.activations.sigmoid,
    tf.keras.activations.tanh,
    tf.keras.activations.relu,
    tf.keras.activations.elu,
    tf.keras.activations.hard_sigmoid,
    tf.keras.activations.leaky_relu,
]

fig, axs = plt.subplots(
    np.ceil(len(activations) / 3).astype(int),
    3,
    figsize=(14, 4 * np.ceil(len(activations) / 3).astype(int)),
)

x = np.linspace(-10, 10, 400)
for i, func in enumerate(activations):
    axs[i // 3, i % 3].plot(x, func(x))
    axs[i // 3, i % 3].set_title(func.__name__)
    axs[i // 3, i % 3].grid(True)

plt.show()

6. Suppose you have an MLP composed of one input layer with 10 passthrough neurons, followed by one hidden layer with 50 artificial neurons, and finally one output layer with 3 artificial neurons. All artificial neurons use the ReLU activation function.
    * a) X.shape = (m, 10) where m is the number of instances
    * b) W_h.shape = (10, 50) & b_h = (50,)
    * c) W_o.shape = (50, 3) & b_o = (3,)
    * d) Y.shape = (m, 3)
    * e) $Y = ReLU(ReLU(XW_h + b_h)W_o + b_o)$
7. Classifying emails into ham and spam is binary classification since the classes are mutually exclusive. 
    * Therefore only **one output neuron** is required, 
    * **sigmoid** would be a good choice for output layer activation since it outputs values between 0 & 1. 

    MNIST is a multiclass classification problem, therefore:
    * you need one output neuron per class, i.e. **10**,
    * **softmax** is an ideal activation function for the output as this returns a probability distribution over the 10 values.

    For predicting housing prices, this is a regression problem,
    * you only need **one output neuron** since you are only predicting one value, and
    * **ReLU** would make sense as it only outputs positive values (passthrough would also work).

8. Backpropagation is how neural networks calculate the gradient of the cost function with respect to the model parameters via the chain rule. Reverse-mode autodiff is the highly computationally efficient method used to calculate the partial derivatives (not just in ANNs).

9. Hyperparameters of a ANN include: **number of hidden layers**, **size of each of the hidden layers**, the **activation function** of each layer, the learning rate, and batch size. If the model is overfitting you should decrease the number of hidden layers & their size, another effective approach is to use early stopping (a.k.a. shrinking the oversized trousers).

## 10. Training an MLP for the MNIST dataset

In [ ]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=10_000, stratify=y_train
)

fig, axs = plt.subplots(5, 20, figsize=(15, 3))
for ax, digit in zip(axs.flatten(), X_train[:100]):
    ax.imshow(digit, cmap="binary")
    ax.axis("off")
plt.show()

print(
    f"X_train shape: {X_train.shape}",
    f"y_train shape: {y_train.shape}\n",
    f"X_val shape: {X_val.shape}",
    f"y_val shape: {y_val.shape}\n",
    f"X_test shape: {X_test.shape}",
    f"y_test shape: {y_test.shape}",
    sep="\n",
)

In [ ]:
def make_model():
    tf.keras.backend.clear_session()  # resets the layer naming

    norm_layer = tf.keras.layers.Normalization()
    norm_layer.adapt(X_train)

    model = tf.keras.Sequential(
        [
            tf.keras.layers.Input((28, 28)),
            norm_layer,
            tf.keras.layers.Flatten(name="flatten"),
            tf.keras.layers.Dense(125, activation="relu"),
            tf.keras.layers.Dense(125, activation="relu"),
            tf.keras.layers.Dense(125, activation="relu"),
            tf.keras.layers.Dense(125, activation="relu"),
            tf.keras.layers.Dense(10, activation="softmax"),
        ]
    )

    model.compile(
        loss="sparse_categorical_crossentropy",
        optimizer=tf.keras.optimizers.Adam(),
        metrics=["accuracy"],
    )

    return model


model = make_model()
model.summary()

tf.keras.utils.plot_model(
    model,
    to_file="models/10_MNIST_MLP.png",
    show_layer_names=True,
    show_shapes=True,
    show_layer_activations=True,
    dpi=50,
)

### Creating a custom callback to find the optimal learning rate

In [ ]:
class ExponentialLearningRate(tf.keras.callbacks.Callback):
    def __init__(self, starting_lr, end_lr, n_iter):
        self.factor = (end_lr / starting_lr) ** (1 / n_iter)
        self.starting_lr = starting_lr
        self.end_lr = end_lr
        self.learning_rates = []
        self.losses = []

    def on_train_begin(self, logs=None):
        self.model.optimizer.learning_rate = self.starting_lr

    def on_train_batch_end(self, batch, logs=None):
        lr = self.model.optimizer.learning_rate.numpy() * self.factor
        if lr > self.end_lr:
            self.model.stop_training = True

        self.model.optimizer.learning_rate = lr
        self.learning_rates.append(lr)
        self.losses.append(logs["loss"])


exponential_learning_rate = ExponentialLearningRate(1e-5, 1e-1, 5_000)

history = model.fit(
    X_train,
    y_train,
    epochs=5,
    validation_data=(X_val, y_val),
    callbacks=[exponential_learning_rate],
)

In [ ]:
fig, ax = plt.subplots()

ax.plot(exponential_learning_rate.learning_rates, exponential_learning_rate.losses)
ax.axvline(x=0.003, color="r", linestyle="--")
ax.set_xscale("log")
ax.set_xlabel("Learning Rate")
ax.set_ylabel("Loss")
ax.grid(True)

plt.show()

From the left hand plot we see that the loss starts to rise again at a learning rate of **0.003**, therefore I set the learning rate to 10 times lower that, before retraining the model.

In [ ]:
model = make_model()

model.optimizer.learning_rate = 3e-4

early_stopping = tf.keras.callbacks.EarlyStopping(
    verbose=1,
    patience=3,
    restore_best_weights=True,
)

history = model.fit(
    X_train,
    y_train,
    epochs=50,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping],
)

In [ ]:
pd.DataFrame(history.history).plot(
    grid=True,
    xlabel="Epoch",
    style=["r--", "r--.", "b-", "b-*"],
)
plt.show()

In [ ]:
model.evaluate(X_test, y_test)

### Using Keras Tuner & TensorBoard

In [ ]:
tf.keras.backend.clear_session()  # resets the layer naming


def build_model(hyper_params):
    num_hidden_layers = hyper_params.Int(
        "num_hidden_layers", min_value=2, max_value=10, default=4
    )
    num_neurons = hyper_params.Int(
        "num_neurons", min_value=16, max_value=512, default=125
    )
    learning_rate = hyper_params.Float(
        "learning_rate", min_value=1e-5, max_value=1e-2, default=3e-4, sampling="log"
    )
    optimizer = hyper_params.Choice("optimizer", values=["sgd", "adam"])
    activation = hyper_params.Choice("activation", values=["relu", "tanh", "sigmoid"])

    model = tf.keras.Sequential()

    model.add(tf.keras.layers.Input((28, 28)))
    norm_layer = tf.keras.layers.Normalization()
    norm_layer.adapt(X_train)
    model.add(norm_layer)
    model.add(tf.keras.layers.Flatten(name="flatten"))
    for _ in range(num_hidden_layers):
        model.add(tf.keras.layers.Dense(num_neurons, activation=activation))
    model.add(tf.keras.layers.Dense(10, activation="softmax"))

    if optimizer == "sgd":
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    elif optimizer == "adam":
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    else:
        print("ERROR: no optimizer set")
        return

    model.compile(
        loss="sparse_categorical_crossentropy",
        optimizer=optimizer,
        metrics=["accuracy"],
    )

    return model


hyperband_tuner = kt.Hyperband(
    build_model,
    objective="val_accuracy",
    max_epochs=10,
    factor=5,
    hyperband_iterations=4,
    overwrite=True,
    directory="models/10/hyperband_tuning",
    project_name="hyperband_tuning",
)


tensorboard_cb = tf.keras.callbacks.TensorBoard(
    "models/10/hyperband_tuning/tensorboard"
)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    verbose=1,
    patience=5,
    restore_best_weights=True,
)

hyperband_tuner.search(
    X_train[:10_000],
    y_train[:10_000],
    validation_data=(X_val, y_val),
    callbacks=[tensorboard_cb, early_stopping_cb],
)

In [ ]:
hyperband_tuner.get_best_hyperparameters()[0].values

### Retraining a model with the best hyper parameters

In [ ]:
model = build_model(hyperband_tuner.get_best_hyperparameters()[0])

tensorboard_cb = tf.keras.callbacks.TensorBoard(
    "models/10/tuned_hyper_params_refit/tensorboard"
)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    verbose=1,
    patience=10,
    restore_best_weights=True,
)

checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    "models/10/tuned_hyper_params_refit/model.weights.h5",
    save_best_only=True,
    save_weights_only=True,
)

history = model.fit(
    X_train,
    y_train,
    epochs=50,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping_cb, checkpoint_cb, tensorboard_cb],
)

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
y_pred = model.predict(X_test).argmax(axis=-1)
misclassified_mask = y_pred != y_test
misclassified_digits = X_test[misclassified_mask]

print(
    f"Misclassified {sum(misclassified_mask):,}/{X_test.shape[0]:,} digits in the test set."
)

fig, axs = plt.subplots(sum(misclassified_mask) // 10, 10, figsize=(15, 70))
for i, ax in enumerate(axs.flatten()):
    ax.imshow(misclassified_digits[i], cmap="binary")
    ax.set_title(
        f"predicted: {y_pred[misclassified_mask][i]}\nactual: {y_test[misclassified_mask][i]}"
    )
    ax.axis("off")
plt.show()